# Clustering images

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from mpl_toolkits.mplot3d import Axes3D

In [ ]:

img = cv2.imread("img1.jpg")/255
scale_percent = 50 # percent of original size
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
  
# resize image
resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
r, g, b = cv2.split(resized)
r = r.flatten()
g = g.flatten()
b = b.flatten()#plotting 
fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(r, g, b)
plt.show()

In [ ]:
img = resized

In [ ]:
#convert the MxNx3 image to a Kx3 image where k = MxN
vectorised = img.reshape((-1,3))

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=0).fit(vectorised)
pic2show = kmeans.cluster_centers_[kmeans.labels_]

In [ ]:
cluster_pic = pic2show.reshape(img.shape[0], img.shape[1], img.shape[2])
plt.imshow(cluster_pic)

In [ ]:
img_df = pd.DataFrame(vectorised)
img_df.rename(columns={0:'R', 1:'G', 2: 'B'}, inplace =True)

In [ ]:
#taking random centroids for initial tests
centroids = img_df.sample(5)

In [ ]:
centroids

In [ ]:
X = img_df

In [ ]:
k = 5
diff = 1
j=0

while(diff!=0):
    XD=X
    i=1
    #iterate over each centroid point 
    for index1,row_c in centroids.iterrows():
        ED=[]
        #iterate over each data point
        print("Calculating distance")
        for index2,row_d in tqdm(XD.iterrows()):
            #calculate distance between current point and centroid
            d1=(row_c["R"]-row_d["R"])**2
            d2=(row_c["G"]-row_d["G"])**2
            d3=(row_c["B"]-row_d["B"])**2
            d=np.sqrt(d1+d2+d3)
            #append disstance in a list 'ED'
            ED.append(d)
        #append distace for a centroid in original data frame
        X[i]=ED
        i=i+1

    C=[]
    print("Getting Centroid")
    for index,row in tqdm(X.iterrows()):
        #get distance from centroid of current data point
        min_dist=row[1]
        pos=1
        #loop to locate the closest centroid to current point
        for i in range(k):
            #if current distance is greater than that of other centroids
            if row[i+1] < min_dist:
                #the smaller distanc becomes the minimum distance 
                min_dist = row[i+1]
                pos=i+1
        C.append(pos)
    #assigning the closest cluster to each data point
    X["Cluster"]=C
    #grouping each cluster by their mean value to create new centroids
    centroids_new = X.groupby(["Cluster"]).mean()[["R","G", "B"]]
    if j == 0:
        diff=1
        j=j+1
    else:
        #check if there is a difference between old and new centroids
        diff = (centroids_new['R'] - centroids['R']).sum() + (centroids_new['G'] - centroids['G']).sum() + (centroids_new['B'] - centroids['B']).sum()
        print(diff.sum())
    centroids = X.groupby(["Cluster"]).mean()[["R","G","B"]]

In [ ]:
centroids = centroids.to_numpy()

In [ ]:
labels = X["Cluster"].to_numpy()
segmented_image = centroids[labels-1]
segmented_image = segmented_image.reshape(img.shape)

In [ ]:
plt.imshow(segmented_image)